In [1]:
import sys
sys.path.insert(1, '/home/jupyter/crisp')

In [2]:
from models.LinearInvariantCausalPrediction import *
from models.TorchLinearInvariantCausalPrediction import *
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [3]:
# from synthetic.synthetic_generator import synthetic_generator
# df = synthetic_generator()
# df.to_pickle("../data/test.pkl")

In [4]:
import pandas as pd
import pickle
with open("../data/test.pkl",'rb') as file:
    df = pickle.load(file)

In [5]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        'dataset_fp': '../data/test.pkl',
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
#         'predictors': ['All'],
        'predictors': list(df.columns[0:10]),
        'environments': ['env_split'],
        'exclude': ['Subj_ID']
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [6]:
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

Running a per sample experiment
Loaded  2  train environments
Env  0  has  33  samples
X shape  (33, 10)  y shape  (33, 1)
Env  1  has  39  samples
X shape  (39, 10)  y shape  (39, 1)
Loaded val set, X shape: (11, 10)  y shape:  (11, 1)
Loaded test set, X shape: (17, 10)  y shape:  (17, 1)


In [7]:
icp_solution = False
# run linear and nonlinear ICP
ICP_args = {
    "max_set_size": 2,
    "alpha": 0.05,
    "seed": 12,
    "verbose": 0
}

# Initialise Linear ICP and run (train and test called internally)
ICPmod = TorchInvariantCausalPrediction(environment_datasets, val_dataset, test_dataset, ICP_args)
# Get results on test_dataset
torch_ICP_results_dict = ICPmod.results()
icp_solution = torch_ICP_results_dict['solution']

# ICP_results_dict['original_feature_indices'] = keep_columns_indices_to_original_features[ICP_results_dict['selected_feature_indices']]

if config['verbose']:
    print(torch_ICP_results_dict)

print("Finished ICP")


Testing  56  permutations with max set size:  2
No intersection, trying defining sets
Pruning defining set trees
Found the defining sets!
{'solution': True, 'intersection': False, 'test_acc': 0.23529411852359772, 'test_nll': tensor(0.7438), 'test_probs': tensor([[-0.1297],
        [-0.0145],
        [-0.5948],
        [-0.6871],
        [-1.0297],
        [-0.0989],
        [ 1.0037],
        [ 0.3690],
        [ 0.6871],
        [ 1.6079],
        [ 0.0169],
        [ 0.2661],
        [-0.0661],
        [-0.0774],
        [-0.3619],
        [-0.0436],
        [-0.3973]]), 'test_labels': tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.]), 'feature_coeffients': array([[ 0.03330465, -0.146639  ,  0.12274949,  0.06694975, -0.26345703,
         0.02608589,  0.26986283,  0.26995686, -0.14337632,  0.17462766]],
      dtype=float32), 'selected_features': array(['Non_causal_0', 'Non_causal_1', 'Non_causal_2', 'Non_causal_3',
       'Non_causal_4', 'Non_causal_5', 'Non

In [10]:
icp_solution = False
# run linear and nonlinear ICP
ICP_args = {
    "max_set_size": 2,
    "alpha": 0.05,
    "seed": 12,
    "verbose": 0
}

# Initialise Linear ICP and run (train and test called internally)
ICPmod = InvariantCausalPrediction(environment_datasets, val_dataset, test_dataset, ICP_args)
# Get results on test_dataset
ICP_results_dict = ICPmod.results()
icp_solution = ICP_results_dict['solution']

# ICP_results_dict['original_feature_indices'] = keep_columns_indices_to_original_features[ICP_results_dict['selected_feature_indices']]

if config['verbose']:
    print(ICP_results_dict)

print("Finished ICP")



Testing  56  permutations with max set size:  2
No intersection, trying defining sets
Pruning defining set trees
Found the defining sets!
{'solution': True, 'intersection': False, 'test_acc': 0.7647058963775635, 'test_nll': tensor(0.5426), 'test_probs': tensor([[0.3965, 0.6035],
        [0.1129, 0.8871],
        [0.5126, 0.4874],
        [0.3247, 0.6753],
        [0.2692, 0.7308],
        [0.2445, 0.7555],
        [0.5088, 0.4912],
        [0.2737, 0.7263],
        [0.7425, 0.2575],
        [0.2586, 0.7414],
        [0.5807, 0.4193],
        [0.4545, 0.5455],
        [0.7371, 0.2629],
        [0.4596, 0.5404],
        [0.3347, 0.6653],
        [0.1863, 0.8137],
        [0.3071, 0.6929]]), 'test_labels': tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.]), 'feature_coeffients': array([ 0.39099739,  0.10415593,  0.        ,  0.01191579, -0.05776088,
        0.        , -0.17892409,  0.56978604,  0.        , -0.18202654]), 'selected_features': array(['Non_causal_0

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [17]:
torch_ICP_results_dict['selected_features']

array(['Non_causal_0', 'Non_causal_1', 'Non_causal_2', 'Non_causal_3',
       'Non_causal_4', 'Non_causal_5', 'Non_causal_6', 'Causal_0',
       'Non_causal_7', 'Non_causal_8'], dtype='<U12')